# 7. Metody specjalne

Metody specjalne (magic methods) to takie, które umożliwiają kontrolę działania różnych wbudowanych mechanizmów języka np. pozwalają przeładować operatory lub zmienić sposób wypisywania obiektów na standardowe wyjście.

In [3]:
import operator


def sieve(n):
    sieve = [True] * (n + 1)
    sieve[0] = sieve[1] = False
    i = 2
    while i * i <= n:
        if sieve[i]:
            k = i * i
            while k <= n:
                sieve[k] = False
                k += i
        i += 1
    return sieve


class PrimeCounter(object):
    
    def __init__(self, start, stop):
        self.start = start
        self.stop = stop
        self.primes = self._count(start, stop)
        
    def _count(self, start, stop):
        values = sieve(stop)
        primes = []
        for i, val in enumerate(values):
            if val and i >= start:
                primes.append(i)
        return primes
    
    def __str__(self):
        return " ".join(["<Primes", str(self.primes), ">"])
    
    def __len__(self):
        return len(self.primes)
    
    def __eq__(self, other):
        return len(self.primes) == len(other.primes)
    
x = PrimeCounter(0, 50)
y = PrimeCounter(0, 10)
z = PrimeCounter(3, 11)
print len(x)
print x
print y, z
print z == y

SyntaxError: invalid syntax (<ipython-input-3-a59f9de87b0e>, line 45)

Inne operatory do przeładowania:

In [64]:
# object.__lt__(self, other)
# object.__le__(self, other)
# object.__ne__(self, other)
# object.__gt__(self, other)
# object.__ge__(self, other)
# object.__add__(self, other) 
# object.__sub__(self, other) 
# ...

## 7.1. Getitem i setitem

In [67]:
my_dict = {'first': 1, 'second': 2}

dir(my_dict)

['__class__',
 '__cmp__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'has_key',
 'items',
 'iteritems',
 'iterkeys',
 'itervalues',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values',
 'viewitems',
 'viewkeys',
 'viewvalues']

In [68]:
my_dict.__getitem__('first')

1

In [69]:
my_dict.__setitem__('third', 3)
print my_dict

{'second': 2, 'third': 3, 'first': 1}


Do czego to może służyć?

In [4]:
from collections import namedtuple

Job = namedtuple('Job', ['name', 'priority'])


class SuperiorHolder(object):
    
    def __init__(self):
        self.lower_priority = {}
        self.higher_priority = {}
        
    def __getitem__(self, item):
        if self.higher_priority.get(item):
            return self.higher_priority[item]
        return self.lower_priority[item]
    
    def __setitem__(self, key, item):
        if item.priority == 1:
            self.higher_priority[key] = item
        elif item.priority == 0:
            self.lower_priority[key] = item
        else:
            raise Exception("Incorrect priority")
            
            
holder = SuperiorHolder()
first = Job('create', 1)
second = Job('delete', 0)
holder[first.name] = first
holder[second.name] = second

print holder['create']
print holder['delete']
# print holder['update']

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-4-60eb8a8deaf4>, line 32)

* Do tworzenia naszych własnych struktur danych (ofc)
* Do ukrywania złożoności tych struktur

## 7.2. Setattr i getattr

Dynamiczne dodawanie atrybutów:

In [79]:
obtained_info = {"state": "OK", "id": 234, "type": "car", "wheels": "OK"}


class Resource(object):
    
    def __init__(self, **kwargs):
        self.id = kwargs['id']


def construct_dynamic(obtained):
    res = Resource(**obtained)
    for key, value in obtained.iteritems():
        setattr(res, key, value)
    return res

res = construct_dynamic(obtained_info)
print res.state

OK


Settattr odpowiada metoda specjalna _ _ setattr_ _

In [84]:
obtained_info = {"state": "BAD", "reason": {"msg": "Unknown"},
                 "id": 235, "type": "car", "wheels": "OK"}

class Resource(object):
    
    def __init__(self, **kwargs):
        self.id = kwargs['id']
        
    def __setattr__(self, key, value):
        if key != "reason":
            super(Resource, self).__setattr__(key, value)
        else:
            super(Resource, self).__setattr__(key, value["msg"])
            
res = Resource(**obtained_info)
setattr(res, "reason", {"msg": "Unknown"})
print res.reason

Unknown


## 7.3. Context manager

Context manager jest to obiekt, który można użyć razem ze słówkiem kluczowym 'with'.

In [85]:
with open('./magic_file', 'r') as rd:
    for line in rd:
        print line

This was loaded from the file!



In [86]:
rd = open('./magic_file', 'r')
dir(rd)

['__class__',
 '__delattr__',
 '__doc__',
 '__enter__',
 '__exit__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__iter__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'close',
 'closed',
 'encoding',
 'errors',
 'fileno',
 'flush',
 'isatty',
 'mode',
 'name',
 'newlines',
 'next',
 'read',
 'readinto',
 'readline',
 'readlines',
 'seek',
 'softspace',
 'tell',
 'truncate',
 'write',
 'writelines',
 'xreadlines']

In [87]:
rd.close()

In [88]:
rd.__enter__

<function __enter__>

In [ ]:
rd.__exit__

Context manager tworzymy implementując metody _ _ enter _ _ i _ _ exit _ _ .

In [19]:
class FatalError(Exception):
    pass


class Connection(object):
    
    INIT = 'init'
    CONNECTED = 'connected'
    DISCONNECTED = 'disconnected'
    
    def __init__(self, **connection_data):
        self._conn_data = connection_data
        self.state = self.INIT
        
    def _establish_db_conn(self):
        print "Connecting to {}.".format(self._conn_data['host'])
        
    def _close_db_conn(self):
        print "Closing connection to {}.".format(self._conn_data['host'])
        
    def run(self, query):
        raise FatalError("It's not going to work.")
        
    def __enter__(self):
        self._establish_db_conn()
        self.state = self.CONNECTED
        return self
        
    def __exit__(self, type, value, traceback):
        print type, value, traceback
        # this will be invoked always
        self._close_db_conn()
        self.state = self.DISCONNECTED
        
        
conn_details = {
    'host': 'darpa01',
    'password': 'Putin',
}

with Connection(**conn_details) as conn:
    print conn.state
    conn.run("select * from *")
    print "This won't be printed"


Connecting to darpa01.
connected
<class '__main__.FatalError'> It's not going to work. <traceback object at 0x7f46668816c8>
Closing connection to darpa01.


FatalError: It's not going to work.

In [14]:
class EnhancedConnection(Connection):
    
    def _log_error(self, type, value, trace):
        print ("Found error during execution: {}:{}."
               " Will close the connection do db.").format(type, value)
        
    def __exit__(self, type, value, traceback):
        if type:
            self._log_error(type, value, traceback)
        self._close_db_conn()
        self.state = self.DISCONNECTED
        
        
with EnhancedConnection(**conn_details) as conn:
    print conn.state
print conn.state

Connecting to darpa01.
connected
Closing connection to darpa01.
disconnected


In [18]:
with EnhancedConnection(**conn_details) as conn:
    print conn.state
    conn.run("oh my query")
print conn.state

Connecting to darpa01.
connected
Found error during execution: <class '__main__.FatalError'>:It's not going to work.. Will close the connection do db.
Closing connection to darpa01.


FatalError: It's not going to work.

Gdy _ _ exit _ _ nie robi tego, czego chcemy (albo metody nie ma w ogóle):

In [29]:
import socket
from contextlib import closing

with closing(socket.socket(socket.AF_INET, socket.SOCK_STREAM)) as sock:
    sock.bind((socket.gethostname(), 1080))
    sock.listen(2)

Context manager przydaje się wszędzie tam, gdzie wykonujemy ustalone czynności na początek i koniec wywołania określonego kodu. Najczęściej nie są one częścią właściwej logiki. Przykłady:
* nawiązanie i zamknięcie połączenia do bazy danych
* otworzenie i zamknięcie pliku
* uzyskanie i zwolnienie semafora
* przydzielenie i zwolnienie wartości z puli
* konstruktory i destruktory

## ZADANIE

Zaimplementuj słownik, który będzie działał poprawnie w poniższych sytuacjach:

In [35]:
class AutoDict(dict):
    pass

ad = AutoDict()
ad["notdeep"] = 1
ad["deeper"]["ok"] = 'OK'
ad["deeper"]["bad"] = 'BAD'
ad[1][2][3][4] = 5

print ad[1][2][3][4]
print ad

KeyError: 'deeper'